# Creating a Service Lab

### Introduction

In this lesson, we'll use Kubernetes to create a deployment with a backend Flask API and a frontend streamlit app.  And then through there, we'll create a new service to allow us to make requests to our Flask API from our streamlit application.  Let's get started.

### Building our Docker Images

If we look at our `services` folder, we can see our backend application inside of the `api` folder.  

```bash
services/api/
├── Dockerfile
├── requirements.txt
└── restaurants
    ├── __init__.py
```

Let's take a look at the Dockerfile used to build the streamlit application.

```Dockerfile
FROM python:3.7
EXPOSE 8501
WORKDIR /usr/src/app

COPY requirements.txt ./
RUN pip install -r requirements.txt

COPY . .

CMD ["streamlit", "run", "src/index.py"]
```

Note that we expose port 8501 as that is where streamlit is located.  And we change the working directory to `/usr/src/app` on the image.  Then we copy over the and install the requirements.txt file, and then copy over the rest of the codebase before running the streamlit command.  

This image can take some time to install, so have it available for you at `jek2141/restaurants_frontend`.  Later on, you can use it to be part of your kubernetes pod.

### Installing the API

```Dockerfile
FROM python:3.7-alpine
WORKDIR /usr/src/app

COPY ./ ./

RUN pip3 install -r requirements.txt

CMD ["flask", "run", "--host=0.0.0.0"]
```

Then comes the image for the API.  This has fewer dependencies.  So let's build this image and tag it as `<username>/restaurants_api`.  

Before we push it up to Dockerhub, confirm that it is working properly.  Expose the proper port, and visit the restaurants route in the browser and we should see the following:

<img src="./restaurants_route.png" width="40%">

Next push this repository to Dockerhub.

### Building our Deployments

Now that we have both of our images available on Dockerhub, let's get deploy them into pods in Kubernetes.  First fill in the `api-deployment.yaml` and `frontend-deployment.yaml` config files in the k8s folder.  Make sure that the ports are setup correctly, and that the correct images are pulled.  Configure the deployments so that two images of each are created.

<img src="./get_pods.png" width="70%">

### Building the Service

Now we need to set up a service so that the api deployment accessible.  We must name it `api-deployment-service`, as that is how it is referenced in the streamlit application.

We can confirm that the service is working, by running `kubectl get services` and viewing the `api-deployment-service`.

<img src="./get_services.png" width="70%">

Let's confirm that the deployment service is connected to the correct pods.  First check the ip addresses of the api pods with the following:

`kubectl describe pods -l component=api`

> The -l stands for label, and we are selecting the pods with the label of api.

Write them down below.

> 172.17.0.5, 172.17.0.4

Then let's examine the virtual ip address of the service as well as the ip addresses of the api pods the traffic is forwarded to.  Confirm that they are the same as written. down above.

### Confirming the connection

Now we won't be able to see our application in the browser yet, as we do not have any service that makes our pods externally available.  But we can check that we can make a request to our Flask API from our frontend pod.  First connect to one of front end pods, and then make a request through curl to the `api-deployment-service`.

> If it works correctly, we should see the following.

<img src="./restaurants.png" width="70%">

### Summary